### 002:  codes for segmentation

Read microscope images in the target directory, split them into patch images and predict VB area using the UNET model.
Generated segmented patches are then reassembled to the images and stored in r_target directory as original_filename+_predicted.png.

In [1]:
from common.data_prep import *
from common.model import *
from common.img_patch import *


# target image for segmentation
target = './_test'

# define your job number under _run directory
job_num = '001'
# set reduction_factor of the initial image 
r_f = 2


set_dir(os.path.join('./_run',job_num,'segmentation')) # segmented VB areas of test images
#set_dir(os.path.join('./_run',job_num,'morphology')) # calculated VB morphological parameters
#set_dir(os.path.join('./_run',job_num,'extracted_VB_images')) # extracted VBs along the radial positions

           
target ='./_test'
r_target = os.path.join('./_run',job_num,'segmentation')


file_list=sorted([filename for filename in os.listdir(target) if not filename.startswith('.')])


for fln in range(len(file_list)):
    img_path=os.path.join(target, file_list[fln])
    res_path=os.path.join(r_target, file_list[fln])
    img=io.imread(img_path, as_gray=True) # gray scale

    # r_f=2 #factor of image reduction
    px=int(img.shape[0]/r_f)
    py=int(img.shape[1]/r_f)
    img_s=trans.resize(img, (px,py))
    # making image patch of 512*512 )
    testdata = patch_Gen(img_s)
    pos = patch_Counter(img_s)
    # predicton of vascular bundles by UNET        
    model = unet()
    model.load_weights(os.path.join('./_run',job_num, 'model','unet_moso_2.hdf5'))
    results = model.predict_generator(testdata,len(pos),verbose=1)
    #saveResult("./prediction",results)
    # reassemble to the image
    rec_img=patch_Assemble(img_s, results)
    rec_img=img_as_ubyte(rec_img)
    #　合成された画像を保存
    cv2.imwrite(res_path.replace('.png', '_predicted.png'),rec_img)

2023-08-19 17:38:06.383809: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-19 17:38:07.898673: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 46717 MB memory:  -> device: 0, name: NVIDIA RTX A6000, pci bus id: 0000:1a:00.0, compute capability: 8.6
2023-08-19 17:38:07.900392: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 46717 MB memory:  -> device: 1, name: NVIDIA RTX A6000, pci bus id: 0000:67:00.0, compute capability: 8.6
2023-08-19 17:38:07.901964: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device

14/14 [==============================] - 0s 11ms/step
